In [47]:
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd

In [48]:
week_df = pd.read_csv('../base_datasets/tracking_week_4.csv')
week_df.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022092900,57,42654.0,La'el Collins,1,2022-09-29 20:16:00.099999,71.0,CIN,left,86.21,30.88,0.00,0.00,0.00,262.60,246.29,NaN
1,2022092900,57,42654.0,La'el Collins,2,2022-09-29 20:16:00.200000,71.0,CIN,left,86.21,30.88,0.00,0.00,0.00,263.32,234.76,NaN
2,2022092900,57,42654.0,La'el Collins,3,2022-09-29 20:16:00.299999,71.0,CIN,left,86.21,30.87,0.01,0.23,0.01,263.32,173.85,NaN
3,2022092900,57,42654.0,La'el Collins,4,2022-09-29 20:16:00.400000,71.0,CIN,left,86.21,30.86,0.07,0.69,0.01,263.32,191.83,NaN
4,2022092900,57,42654.0,La'el Collins,5,2022-09-29 20:16:00.500000,71.0,CIN,left,86.21,30.85,0.19,1.01,0.02,263.32,192.85,NaN


In [49]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(week_df):
    # Replace missing values with 0 in columns: 'jerseyNumber', 'nflId'
    week_df = week_df.fillna({'jerseyNumber': 0, 'nflId': 0})
    # Change column type to string for column: 'nflId'
    week_df = week_df.astype({'nflId': 'string'})
    # Replace all instances of ".0" with "" in column: 'nflId'
    week_df['nflId'] = week_df['nflId'].str.replace(".0", "", case=False, regex=False)
    # Change column type to string for column: 'jerseyNumber'
    week_df = week_df.astype({'jerseyNumber': 'string'})
    # Replace all instances of ".0" with "" in column: 'jerseyNumber'
    week_df['jerseyNumber'] = week_df['jerseyNumber'].str.replace(".0", "", case=False, regex=False)
    # Change column type to string for columns: 'club', 'playDirection', 'event', 'displayName', 'time'
    week_df = week_df.astype({'club': 'string', 'playDirection': 'string','event': 'string','displayName': 'string'})
    # Reduce the floats to two decimal points
    week_df[['x', 'y', 's', 'a', 'dis']] = week_df[['x', 'y', 's', 'a', 'dis']].round(2)
    # Change column type to datetime64[ns] for column: 'time'
    week_df = week_df.astype({'time': 'datetime64[ns]'})
    return week_df

week_df_clean = clean_data(week_df.copy())
week_df_clean.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022092900,57,42654,La'el Collins,1,2022-09-29 20:16:00.099999,71,CIN,left,86.21,30.88,0.00,0.00,0.00,262.60,246.29,<NA>
1,2022092900,57,42654,La'el Collins,2,2022-09-29 20:16:00.200000,71,CIN,left,86.21,30.88,0.00,0.00,0.00,263.32,234.76,<NA>
2,2022092900,57,42654,La'el Collins,3,2022-09-29 20:16:00.299999,71,CIN,left,86.21,30.87,0.01,0.23,0.01,263.32,173.85,<NA>
3,2022092900,57,42654,La'el Collins,4,2022-09-29 20:16:00.400000,71,CIN,left,86.21,30.86,0.07,0.69,0.01,263.32,191.83,<NA>
4,2022092900,57,42654,La'el Collins,5,2022-09-29 20:16:00.500000,71,CIN,left,86.21,30.85,0.19,1.01,0.02,263.32,192.85,<NA>


In [50]:
week_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1418686 entries, 0 to 1418685
Data columns (total 17 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   gameId         1418686 non-null  int64         
 1   playId         1418686 non-null  int64         
 2   nflId          1418686 non-null  string        
 3   displayName    1418686 non-null  string        
 4   frameId        1418686 non-null  int64         
 5   time           1418686 non-null  datetime64[ns]
 6   jerseyNumber   1418686 non-null  string        
 7   club           1418686 non-null  string        
 8   playDirection  1418686 non-null  string        
 9   x              1418686 non-null  float64       
 10  y              1418686 non-null  float64       
 11  s              1418686 non-null  float64       
 12  a              1418686 non-null  float64       
 13  dis            1418686 non-null  float64       
 14  o              1357361 non-null  f

In [51]:
unique_values=week_df_clean['gameId'].unique()
print(unique_values)

[2022092900 2022100200 2022100201 2022100202 2022100203 2022100204
 2022100205 2022100206 2022100207 2022100208 2022100209 2022100210
 2022100211 2022100212 2022100213 2022100300]


In [52]:
cin_mia_df = week_df_clean[week_df_clean['gameId'] == 2022092900] 
no_min_df = week_df_clean[week_df_clean['gameId'] == 2022100200]
atl_cle_df = week_df_clean[week_df_clean['gameId'] == 2022100201]
buf_bal_df = week_df_clean[week_df_clean['gameId'] == 2022100202]
ari_car_df = week_df_clean[week_df_clean['gameId'] == 2022100203] 
dal_was_df = week_df_clean[week_df_clean['gameId'] == 2022100204]
det_sea_df = week_df_clean[week_df_clean['gameId'] == 2022100205]
lac_hou_df = week_df_clean[week_df_clean['gameId'] == 2022100206]
ind_ten_df = week_df_clean[week_df_clean['gameId'] == 2022100207]
chi_nyg_df = week_df_clean[week_df_clean['gameId'] == 2022100208]
phi_jax_df = week_df_clean[week_df_clean['gameId'] == 2022100209]
nyj_pit_df = week_df_clean[week_df_clean['gameId'] == 2022100210]
ne_gb_df = week_df_clean[week_df_clean['gameId'] == 2022100211]
den_lv_df = week_df_clean[week_df_clean['gameId'] == 2022100212]
tb_kc_df = week_df_clean[week_df_clean['gameId'] == 2022100213]
la_sf_df = week_df_clean[week_df_clean['gameId'] == 2022100300]

unique_values=week_df_['club'].unique()
print(unique_values)




<StringArray>
['LA', 'SF', 'football']
Length: 3, dtype: string


In [53]:
atl_cle_df.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
182574,2022100201,63,38592,Casey Hayward,1,2022-10-02 13:03:56.400000,29,ATL,left,70.01,35.13,5.39,0.93,0.54,221.36,291.47,<NA>
182575,2022100201,63,38592,Casey Hayward,2,2022-10-02 13:03:56.500000,29,ATL,left,69.51,35.33,5.43,0.82,0.54,218.01,292.77,<NA>
182576,2022100201,63,38592,Casey Hayward,3,2022-10-02 13:03:56.599999,29,ATL,left,69.01,35.55,5.38,0.96,0.54,216.88,294.00,<NA>
182577,2022100201,63,38592,Casey Hayward,4,2022-10-02 13:03:56.700000,29,ATL,left,68.53,35.78,5.34,1.17,0.54,215.32,295.40,<NA>
182578,2022100201,63,38592,Casey Hayward,5,2022-10-02 13:03:56.799999,29,ATL,left,68.05,36.01,5.29,1.31,0.53,214.74,296.49,pass_arrived


In [54]:
unique_values=week_df_['event'].unique()
print(unique_values)

<StringArray>
[                 <NA>,           'ball_snap',             'handoff',
       'first_contact',              'tackle',        'pass_arrived',
 'pass_outcome_caught',       'out_of_bounds',                 'run',
            'qb_slide',       'man_in_motion',               'shift',
           'touchdown',         'play_action']
Length: 14, dtype: string


In [59]:
td_df = atl_cle_df[atl_cle_df['event'] == 'touchdown']
td_df.head()


,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
203082,2022100201,778,39975,Cordarrelle Patterson,62,2022-10-02 13:28:19.099999,84,ATL,right,108.10,52.43,9.62,1.38,0.97,57.01,49.65,touchdown
203148,2022100201,778,41232,Jake Matthews,62,2022-10-02 13:28:19.099999,70,ATL,right,105.98,34.50,3.39,0.88,0.34,90.83,16.38,touchdown
203214,2022100201,778,42345,Marcus Mariota,62,2022-10-02 13:28:19.099999,1,ATL,right,94.11,36.72,1.41,0.35,0.14,60.96,53.18,touchdown
203280,2022100201,778,44903,John Johnson,62,2022-10-02 13:28:19.099999,43,CLE,right,110.12,46.41,7.07,2.18,0.72,7.92,19.73,touchdown
203346,2022100201,778,45038,Isaac Rochell,62,2022-10-02 13:28:19.099999,98,CLE,right,106.63,44.15,5.75,2.81,0.60,31.47,46.90,touchdown


In [60]:
unique_values=td_df['playId'].unique()
print(unique_values)

[ 778 1197 2864]
